# Mustererkennung/Machine Learning - Assignment 8



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
class Classifier:
    
    def accuracy(self, labels, predictions):
        return np.mean(labels == predictions)


data = pd.read_csv("./Data/iris.data", header=None)
data.head(n=5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#### Splitting the data into training/test and according to their class memberships

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data[list(range(4))], data[4], test_size=0.2, random_state=None, stratify=data[4])

X_train_setosa = X_train[y_train=='Iris-setosa'].to_numpy()
X_train_versicolor = X_train[y_train=='Iris-versicolor'].to_numpy()
X_train_virginica = X_train[y_train=='Iris-virginica'].to_numpy()

y_train_setosa = y_train[y_train=='Iris-setosa'].to_numpy()
y_train_versicolor = y_train[y_train=='Iris-versicolor'].to_numpy()
y_train_virginica = y_train[y_train=='Iris-virginica'].to_numpy()

X_test_setosa_v_v = X_test.to_numpy()
y_test_setosa_v_v = (y_test == 'Iris-setosa').astype(int).to_numpy()

X_test_versicolor_virginica = X_test[y_test!='Iris-setosa'].to_numpy()
y_test_versicolor_virginica = (y_test[y_test!='Iris-setosa'] == 'Iris-versicolor').astype(int).to_numpy()

In [16]:
X_train.shape

(120, 4)

In [27]:
X_train_setosa.shape

(40, 4)

In [29]:
X_test_setosa_v_v.shape

(30, 4)

In [33]:
y_test_setosa_v_v

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1])

In [32]:
y_test_versicolor_virginica.shape

(20,)

In [20]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

## Task 1:  seperate Setosa from Versicolor and Virginica 

In [52]:
def sub_norm(w_1, w_2):
    return np.linalg.norm(w_1-w_2)

In [201]:
class Perceptron:
    def __init__(self, w, theta):
        self.w = w
        self.theta = theta
    
    def fit(self, X_pos, X_neg, y_pos, y_neg):
        # initialize the w
        w_1 = np.mean(X_pos, 0)
        
        X_data = np.concatenate((X_pos, X_neg), axis=0)
        
        n_data = X_data.shape[0]
        n_pos = X_pos.shape[0]
        
        for i in range(10000): 
            
            w = w_1
            
            index = np.random.randint(n_data)
            v = X_data[index]
            
            if(index < n_pos and w.T @ v > 0):
                continue
            
            if(index < n_pos and w.T @ v <= 0):
                w_1 = w + v
            
            if(index >= n_pos and w.T @ v < 0):
                continue
            
            if(index >= n_pos and w.T @ v >= 0):
                w_1 = w - v
            
            if(sub_norm(w_1, w) <= self.theta):
                break
        self.w = w_1
        
    def calculate(self, x):
        if(self.w.T @ x > 0):
            return 1
        else:
            return 0
        
    def predict(self, X_data):
        num = X_data.shape[0]
        res = np.zeros(num)
        
        for i in range(num):
            if(self.calculate(X_data[i]) == 0):
                res[i] = 1
                
        return res
        
    def accuracy(self, labels, predictions):
        return np.mean(labels == predictions)
        

In [202]:
perceptron = Perceptron(np.mean(X_train_setosa, 0),1e-3)

In [203]:
perceptron.fit(X_train_setosa, np.concatenate((X_train_versicolor, X_train_virginica)), y_train_setosa, np.concatenate((y_train_versicolor, y_train_virginica)))
predictions = perceptron.predict(X_test_setosa_v_v)
print(perceptron.accuracy(y_test_setosa_v_v, predictions))

0.6666666666666666


In [204]:
def sub_vec(X_data, v):
    for i in range(len(X_data)):
        X_data[i] -= v
    return X_data

In [205]:
X_test_versicolor = X_test[y_test=='Iris-versicolor'].to_numpy()
y_test_versicolor = y_test[y_test=='Iris-versicolor'].to_numpy()

temp = np.mean(X_train, 0)

perceptron.fit(X_train_versicolor, X_train_virginica,y_train_versicolor, y_train_virginica)
predictions = perceptron.predict(X_test_versicolor_virginica)
print(perceptron.accuracy(y_test_versicolor_virginica, predictions))

0.5


__The problem while classifying Versicolor from Virginica__ is that all vector have positive projection on w so that with perceptron we can't get right separation. 

# Task 2 MLP

In [149]:
from sklearn.linear_model import Perceptron